# 텍스트 분류 실습 - 20 뉴스그룹 분류
    - 텍스트 분류: 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것

### 1. 텍스트 정규화


In [29]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

# 파이썬 딕셔너리와 유사한 Bunch 객체를 반환한다.
news_data = fetch_20newsgroups(subset='all', random_state=156)

In [30]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [31]:
# Target 클래스 구성요소 확인

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print()
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [11]:
# 개별 데이터가 텍스트로 어떻게 구성돼 있는지 데이터 한 개만 추출해 값을 확인
# 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있음 확인
# 내용을 제외하고 제목 등의 다른 정보는 제거할 것
#   순수한 텍스트만으로 구성된 기사 내용으로 어떤 뉴스그룹에 속하는지 분류할 것
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [38]:
# subset 인자를 전달하여 train과 test 데이터를 각각 불러올 수 있다.
# remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)

X_train = train_news.data
y_train = train_news.target

In [39]:
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)

X_test = test_news.data
y_test = test_news.target

In [40]:
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


### 2. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

- 11314개의 뉴스그룹 문서가 리스트 형태로 주어짐
- 7532개의 문서가 리스트 형태로 주어짐
- 테스트 데이터에서 CountVectorizer를 적용할 때는 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환해야 한다.
    - 학습 시 설정된 CountVectorizer의 피처 개수 == 테스트 데이터를 CountVectorizer로 변환할 피처 개수
        - 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform()을 이용해 변환한다.
- CountVectorizer.fit_transform(테스트데이터)을 테스트 데이터 세트에 적용할 시 문제점
    - 테스트 데이터 기반으로 다시 CountVectorizer가 fit()을 수행하고 transform()하기 때문에 학습 시 사용된 피처 개수 != 예측 시 사용할 피처 개수

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
# fit(): 훈련을 위한 기본 정보 설정
# transform(): fit()을 통해 설정된 정보를 이용해서 변환
#################### 피처 벡터화 관련###############
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


--------------
491p. 로지스틱 회귀 적용할 것